#Analisis Data Prediksi Cuaca dengan Komputasi Paralel

**Pendahuluan**

Prediksi cuaca melibatkan analisis data dalam jumlah besar, seperti suhu, kelembapan, tekanan udara, dan kecepatan angin. Analisis ini sering menggunakan model regresi atau pembelajaran mesin untuk memprediksi parameter cuaca di masa depan. Namun, proses pengolahan data tersebut membutuhkan waktu yang lama, terutama jika dilakukan secara serial. Oleh karena itu, pendekatan komputasi paralel, termasuk OpenMP, MPI, dan Spark, dapat digunakan untuk mempercepat proses ini.

Tujuan eksperimen ini adalah membandingkan performa komputasi serial, paralel biasa, OpenMP, MPI, dan Spark dalam melakukan prediksi suhu menggunakan regresi linear sederhana pada dataset besar.

**Dataset**
Dataset cuaca yang digunakan adalah simulasi data suhu dengan parameter:

Fitur: Waktu (jam), kelembapan, tekanan udara, kecepatan angin.

Target: Suhu (dalam derajat Celcius).

Dataset akan dihasilkan secara acak untuk keperluan eksperimen ini.

**1. Komputasi Serial**

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
import time

# Generate synthetic weather data
np.random.seed(42)
n_samples = 10**6

data = pd.DataFrame({
    'hour': np.random.randint(0, 24, n_samples),
    'humidity': np.random.uniform(30, 90, n_samples),
    'pressure': np.random.uniform(1000, 1020, n_samples),
    'wind_speed': np.random.uniform(0, 15, n_samples),
    'temperature': np.random.uniform(15, 35, n_samples)
})

# Split features and target
X = data[['hour', 'humidity', 'pressure', 'wind_speed']]
y = data['temperature']

# Komputasi serial
start_time = time.time()
model = LinearRegression()
model.fit(X, y)
serial_time = time.time() - start_time

print(f"Komputasi serial selesai dalam waktu: {serial_time:.2f} detik")

Komputasi serial selesai dalam waktu: 0.11 detik


**2. Komputasi Paralel Biasa**

Menggunakan pustaka Joblib untuk melakukan paralelisme sederhana.

In [4]:
from joblib import Parallel, delayed

# Fungsi paralel untuk fitting model per partisi data
def parallel_fit(partition):
    model = LinearRegression()
    model.fit(partition["X"], partition["y"])
    return model

# Membagi dataset menjadi partisi
n_partitions = 4
partitions = [
    {
        "X": X.iloc[i::n_partitions],
        "y": y.iloc[i::n_partitions]
    }
    for i in range(n_partitions)
]

# Komputasi paralel
start_time = time.time()
models = Parallel(n_jobs=n_partitions)(delayed(parallel_fit)(partition) for partition in partitions)
parallel_time = time.time() - start_time

print(f"Komputasi paralel selesai dalam waktu: {parallel_time:.2f} detik")

ValueError: cannot set WRITEABLE flag to True of this array

**3.  Komputasi Paralel dengan OpenMP dan MPI**

Pendekatan ini memerlukan implementasi di lingkungan HPC, namun untuk simulasi, kita dapat menggunakan pustaka Python seperti mpi4py.

In [ ]:
pip install mpi4py

In [2]:
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

# Membagi data berdasarkan rank
chunk_size = n_samples // size
start_idx = rank * chunk_size
end_idx = (rank + 1) * chunk_size if rank != size - 1 else n_samples

X_chunk = X.iloc[start_idx:end_idx]
y_chunk = y.iloc[start_idx:end_idx]

# Fitting model pada setiap proses
local_model = LinearRegression()
local_model.fit(X_chunk, y_chunk)

# Mengumpulkan model di root process
all_models = comm.gather(local_model, root=0)

if rank == 0:
    print(f"Model dari semua proses dikumpulkan: {len(all_models)}")
    # Menampilkan hasil prediksi dari model pertama sebagai contoh
    if all_models:
        sample_prediction = all_models[0].predict(X.iloc[:5])
        print(f"Hasil prediksi contoh dari model pertama: {sample_prediction}")

RuntimeError: cannot load MPI library
Could not find module 'C:\Users\62822\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\DLLs' (or one of its dependencies). Try using the full path with constructor syntax.
Could not find module 'C:\Users\62822\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Library\bin' (or one of its dependencies). Try using the full path with constructor syntax.
Could not find module 'C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2288.0_x64__qbz5n2kfra8p0\DLLs\impi.dll' (or one of its dependencies). Try using the full path with constructor syntax.
Could not find module 'C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2288.0_x64__qbz5n2kfra8p0\DLLs\msmpi.dll' (or one of its dependencies). Try using the full path with constructor syntax.
Could not find module 'C:\Program Files\WindowsApps\PythonSoftwareFoundation.Pyt

**4. Komputasi Paralel dengan Spark**

Menggunakan PySpark untuk menjalankan operasi secara terdistribusi.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler

# Inisialisasi Spark
spark = SparkSession.builder.appName("Weather Prediction").getOrCreate()

# Konversi DataFrame ke Spark DataFrame
spark_df = spark.createDataFrame(data)

# Menyusun fitur menjadi satu kolom
feature_cols = ['hour', 'humidity', 'pressure', 'wind_speed']
assembler = VectorAssembler(inputCols=feature_cols, outputCol='features')
spark_df = assembler.transform(spark_df)

# Membagi data menjadi training dan test
train_df, test_df = spark_df.randomSplit([0.8, 0.2], seed=42)

# Membuat dan melatih model regresi linear
lr = LinearRegression(featuresCol='features', labelCol='temperature')
model = lr.fit(train_df)

# Evaluasi model
predictions = model.transform(test_df)

# Menampilkan hasil prediksi pertama untuk memastikan hasilnya
print("Menampilkan contoh hasil prediksi:")
predictions.select('temperature', 'prediction').show(5)

spark.stop()


# Tugas.

Lakukan simulasi contoh kode diatas sebagai implementasi dari komputasi paralel. Setelah itu lakukan analisis hasil dari percobaan tersebut, selain melakukan analisis dan pembahasan hasil, lakukan pula analisis dan pembahasan dari kode program dalam melakukan komputasi paralel. Coba pula lakukan pada simulasi pada data yang lebih besar (bisa mencari pentunjuk di internet). Cari kelebihan dan kekurangan dari masing-masing metode dalam penerapan simulasi tersebut. Tuangkan dalam format draft artikel yang terdiri dari:
1. Pendahuluan
2. Penelitian terkait dan landasan teori
3. Meodologi Penelitian
4. Hasil dan Pembahasan
5. Kesimpulan
